In [1]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ProgressMeter",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Pkg.develop(path="../../..")
import Mycelia

  Activating environment at `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Manifest.toml`


In [2]:
function download_and_filter_reads(;outdir="", srr_identifier="")
    
    forward_reads = joinpath(outdir, "$(srr_identifier)_1.fastq")
    reverse_reads = joinpath(outdir, "$(srr_identifier)_2.fastq")
    forward_reads_gz = forward_reads * ".gz"
    reverse_reads_gz = reverse_reads * ".gz"
    trimmed_forward_reads = joinpath(outdir, "trim_galore", "$(srr_identifier)_1_val_1.fq.gz")
    trimmed_reverse_reads = joinpath(outdir, "trim_galore", "$(srr_identifier)_2_val_2.fq.gz")

    if !(isfile(trimmed_forward_reads) && isfile(trimmed_reverse_reads))
        fasterq_dump(outdir=outdir, srr_identifier=srr_identifier)
        trim_galore(outdir=outdir, identifier=srr_identifier)
    end
    isfile(forward_reads_gz) && rm(forward_reads_gz)
    isfile(reverse_reads_gz) && rm(reverse_reads_gz)
end

download_and_filter_reads (generic function with 1 method)

In [3]:
function fasterq_dump(;outdir="", srr_identifier="")
    
    forward_reads = joinpath(outdir, "$(srr_identifier)_1.fastq")
    reverse_reads = joinpath(outdir, "$(srr_identifier)_2.fastq")
    
    forward_reads_gz = forward_reads * ".gz"
    reverse_reads_gz = reverse_reads * ".gz"
    
    if !isfile(forward_reads_gz) && !isfile(reverse_reads_gz)
        fasterq_dump_cmd = `
            conda run --live-stream -n viral-pangenome-discovery
            fasterq-dump
                --outdir $(outdir)
                --mem 1G
                --split-3
                --threads $(Sys.CPU_THREADS)
                --skip-technical
                $(srr_identifier)`
        @time run(fasterq_dump_cmd)
        run(`pigz $(forward_reads)`)
        run(`pigz $(reverse_reads)`)
    else
        @info "$(forward_reads_gz) & $(reverse_reads_gz) already present"
    end
end

fasterq_dump (generic function with 1 method)

In [4]:
function trim_galore(;outdir="", identifier="")
    
    trim_galore_dir = joinpath(outdir, "trim_galore")
    
    forward_reads = joinpath(outdir, "$(identifier)_1.fastq.gz")
    reverse_reads = joinpath(outdir, "$(identifier)_2.fastq.gz")
    
    trimmed_forward_reads = joinpath(trim_galore_dir, "$(identifier)_1_val_1.fq.gz")
    trimmed_reverse_reads = joinpath(trim_galore_dir, "$(identifier)_2_val_2.fq.gz")
    
    if !isfile(trimmed_forward_reads) && !isfile(trimmed_reverse_reads)
        cmd = `conda run --live-stream -n viral-pangenome-discovery trim_galore --cores $(Sys.CPU_THREADS) --output_dir $(trim_galore_dir) --paired $(forward_reads) $(reverse_reads)`
        # cmd = `conda run -n viral-pangenome-discovery trim_galore --output_dir $(trim_galore_dir) --paired $(forward_reads) $(reverse_reads)`
        run(cmd)
    else
        @info "$(trimmed_forward_reads) & $(trimmed_reverse_reads) already present"
    end
end

trim_galore (generic function with 1 method)

In [5]:
srr_list = "$(dirname(pwd()))/metadata/exposome/SraAccList.txt"
srr_identifiers = readlines(srr_list)

594-element Vector{String}:
 "SRR6399459"
 "SRR6399460"
 "SRR6399461"
 "SRR6399462"
 "SRR6399463"
 "SRR6399464"
 "SRR6399465"
 "SRR6399466"
 "SRR6399467"
 "SRR6399468"
 "SRR6399469"
 "SRR6399470"
 "SRR6399471"
 ⋮
 "SRR7365474"
 "SRR7365475"
 "SRR7365476"
 "SRR7365477"
 "SRR7365478"
 "SRR7365479"
 "SRR7365480"
 "SRR7365481"
 "SRR7365482"
 "SRR7365483"
 "SRR7365484"
 "SRR7365485"

In [ ]:
ProgressMeter.@showprogress for srr_identifier in srr_identifiers
    sample_outdir = "$(dirname(pwd()))/data/SRA/$(srr_identifier)"
    # fasterq_dump(outdir=sample_outdir, srr_identifier=srr_identifier)
    download_and_filter_reads(outdir=sample_outdir, srr_identifier=srr_identifier)
end

Progress:   0%|▏                                        |  ETA: 0:06:38spots read      : 75,614,307
reads read      : 151,228,614
reads written   : 151,228,614



4503.502431 seconds (138.66 k allocations: 3.946 MiB)
pigz 2.6
igzip command line interface 2.30.0



Using an excessive number of cores has a diminishing return! It is recommended not to exceed 8 cores per trimming process (you asked for 48 cores). Please consider re-specifying
Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.2
Could not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)
Letting the (modified) Cutadapt deal with the Python version instead
Parallel gzip (pigz) detected. Proceeding with multicore (de)compression using 48 cores

igzip detected. Using igzip for decompressing

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Output directory /home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399464/trim_galore/ doesn't exist, creating it for you...

Output will be written into the directory: /home/jovyan/work/Mycelia/projects/viral-pangenome-disc

2531.293895 seconds (77.99 k allocations: 2.220 MiB)


spots read      : 39,660,282
reads read      : 79,320,564
reads written   : 79,320,564



...

Progress:   5%|█▉                                       |  ETA: 17.72 daysUsing an excessive number of cores has a diminishing return! It is recommended not to exceed 8 cores per trimming process (you asked for 48 cores). Please consider re-specifying
Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.2
Could not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)
Letting the (modified) Cutadapt deal with the Python version instead
Parallel gzip (pigz) detected. Proceeding with multicore (de)compression using 48 cores

igzip detected. Using igzip for decompressing

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Output directory /home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399486/trim_galore/ doesn't exist, creating it for you...

Output will be written i

pigz 2.6
igzip command line interface 2.30.0



Progress:   5%|█▉                                       |  ETA: 18.34 days

2890.503111 seconds (89.06 k allocations: 2.535 MiB)


spots read      : 70,466,130
reads read      : 140,932,260
reads written   : 140,932,260

